In [9]:
import torch
import torchvision
from torchvision import transforms, datasets

In [10]:
train = datasets.MNIST('', train=True, download=True, transform=transforms.Compose([ transforms.ToTensor()]))

test = datasets.MNIST('', train=False, download=True, transform=transforms.Compose([ transforms.ToTensor()]))

trainset = torch.utils.data.DataLoader(train, batch_size=10, shuffle=True)
testset = torch.utils.data.DataLoader(test, batch_size=10, shuffle=False)

### Now we will start building the Neural Network

In [11]:
import torch.nn as nn
import torch.nn.functional as F

In [12]:
# we define a class called Net that inherits from PyToch's Module class 
class Net(nn.Module):
    # this is the constructor
    def __init__(self):
        super().__init__() # call the Module's constructor
        # define a variable fc1 (first fully connected layer) which takes a 28* 28 values as input (each image is 28 pixle by 28 pixle) and outputs 64 values
        self.fc1 = nn.Linear(28*28, 64)
        # define a senond fully connected layer which takes 64 values as input an onputs 64 ones
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 64)
        # this is the output layer as it outputs 10 values (digits from 0 to 9)
        self.fc4 = nn.Linear(64, 10)
        
    # a simple feed forward function using Rectified Linear function as an activation function
    # the x is the input data
    def forward(self, x):
        x = F.relu(self.fc1(x)) # feed the output from the first hidden layer to a Rectified Linear function
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        # feed the output of the fourth layer to a softmax function
        return F.log_softmax(x, dim=1)
    
net = Net()

### Now we will define the optimizer

In [13]:
import torch.optim as optim

# the net.parameters() indicates to the optmizer all the tunable parameters in our NN 
optimizer = optim.Adam(net.parameters(), lr=0.001) 

### Now we will iterate through our data 3 times, and let the optimizer calculate the loss and perform backpropagation to tune the NN parameters

In [14]:
for epoch in range(3): # 3 full passes over the data
    for data in trainset:  # `data` is a batch of data
        X, y = data  # X is the batch of features, y is the batch of targets.
        net.zero_grad()  # sets gradients to 0 before loss calc.
        output = net(X.view(-1, 28 * 28 ))  # pass in the reshaped batch (recall they are 28x28 atm)
        loss = F.nll_loss(output, y)  # calc and grab the loss value
        loss.backward()  # compute the derivative of the loss w.r.t. the parameters
        optimizer.step()  # attempt to optimize weights to account for loss/gradients
    print(loss)  # print loss. We hope loss (a measure of wrong-ness) declines! 

tensor(0.0032, grad_fn=<NllLossBackward>)
tensor(0.0072, grad_fn=<NllLossBackward>)
tensor(0.0014, grad_fn=<NllLossBackward>)


### Now we will test our model with some test data samples and calculate the accuracy of the model

In [15]:
correct = 0
total = 0

# this tells PyTorch to not calculate gradients
with torch.no_grad():
    for data in testset:
        X, y = data
        output = net(X.view(-1, 28 * 28 ))
        for idx, i in enumerate(output):
            if torch.argmax(i) == y[idx]:
                correct += 1
            total += 1

print("Accuracy: ", round(correct/total, 3))

Accuracy:  0.97
